## Deployment for the gNB with dashboard

Until this date, only the Monolithic mode works with the dashboard.

**Step 1:** open a new terminal using ctrl+Alt+T and type:

sudo su

cd /home/dco1020/openairinterface5g/cmake_targets/ran_build/build/

sudo RFSIMULATOR=server ./nr-softmodem --rfsim --sa -O ../../../targets/PROJECTS/GENERIC-NR-5GC/CONF/gnb.sa.band78.fr1.106PRB.usrpb210.conf --gNBs.[0].min_rxtxtime 6 --continuous-tx -d 

![gNB_config_07](../FIGS/H02_5GRAN_UNI_III/gNB_config_07.png)

NRscope graphs ([Reference](https://openairinterface.org/wp-content/uploads/2022/06/OAI-Webinar-Series-Chapter-Four-Presentation-Slides.pdf))

![nrscope](../FIGS/H02_5GRAN_UNI_III/nrscope.png)

**Step 2**: To check the connection of gNodeB to the Core, see AMF logs. Open another terminal (the third one) using ctrl+Alt+T and type: 

docker logs -f oai-amf

The expected return is shown in figure below, where gNodeB shows the connected status.

![gNB_config_06](../FIGS/H02_5GRAN_UNI_III/gNB_config_06.png)



## Deployment for the UE with dashboard

OAI-nrUE deployment with dashboard.

**Step 1:** open a new terminal using ctrl+Alt+T and type:

sudo su

cd /home/dco1020/openairinterface5g/cmake_targets/ran_build/build/

sudo RFSIMULATOR=192.168.70.129 ./nr-uesoftmodem -r 106 --numerology 1 --band 78 -C 3619200000 --rfsim --sa --nokrnmod -O ../../../targets/PROJECTS/GENERIC-NR-5GC/CONF/ue.conf -d

![gNB_config_06](../FIGS/H03_5GUE_UNI_III/UE_config_07.png)

UEscope graphs ([Reference](https://openairinterface.org/wp-content/uploads/2022/06/OAI-Webinar-Series-Chapter-Four-Presentation-Slides.pdf))

![uescope](../FIGS/H03_5GUE_UNI_III/uescope.png)

## Channel Simulation with dashboard

Tutorial made based on the [link](https://gitlab.eurecom.fr/oai/openairinterface5g/-/blob/develop/openair1/SIMULATION/TOOLS/DOC/channel_simulation.md).

**Step 1:** Create the channel simulation file. 

cd /home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/

touch channelmod_rfsimu.conf

**Step 2:** Open the channelmod_rfsimu.conf and add the following lines: 

```
channelmod = {
  max_chan = 10;
  modellist = "modellist_rfsimu_1";
  modellist_rfsimu_1 = (
    {
      model_name     = "moveis_gNB0"
      type           = "AWGN";
      ploss_dB       = 20;
      noise_power_dB = -4;
      forgetfact     = 0;
      offset         = 0;
      ds_tdl         = 0;
    },
    {
      model_name     = "moveis_ue0"
      type           = "AWGN";
      ploss_dB       = 20;
      noise_power_dB = -2;
      forgetfact     = 0;
      offset         = 0;
      ds_tdl         = 0;
    }
  );
};
```

These lines are the configuration parameters for the gNB and UE channels.

**Step 3:** Open the /home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/gnb.sa.band78.fr1.106PRB.usrpb210.conf

cd /home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/gnb.sa.band78.fr1.106PRB.usrpb210.conf

**Step 4:** Add the line below in the end of gnb.sa.band78.fr1.106PRB.usrpb210.conf.

@include "channelmod_rfsimu.conf"

**Step 5:** Open the file /home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/ue.conf 

cd /home/dco1020/openairinterface5g/targets/PROJECTS/GENERIC-NR-5GC/CONF/ue.conf


**Step 6:** Add the following lines in the end of ue.conf.

```
channelmod = {
  max_chan = 10;
  modellist = "modellist_rfsimu_1";
  modellist_rfsimu_1 = (
    {
      model_name     = "moveis_gNB0"
      type           = "AWGN";
      ploss_dB       = 20;
      noise_power_dB = -4;
      forgetfact     = 0;
      offset         = 0;
      ds_tdl         = 0;
    },
    {
      model_name     = "moveis_ue0"
      type           = "AWGN";
      ploss_dB       = 20;
      noise_power_dB = -2;
      forgetfact     = 0;
      offset         = 0;
      ds_tdl         = 0;
    }
  );
};
```


**Step 7:** Deploy OAI gNB:

sudo ./nr-softmodem -O ../../../targets/PROJECTS/GENERIC-NR-5GC/CONF/gnb.sa.band78.fr1.106PRB.usrpb210.conf --gNBs.[0].min_rxtxtime 6 --rfsim --sa --rfsimulator.options chanmod --telnetsrv --channelmod.modellist modellist_rfsimu_1 -d

**Step 8:** Deploy OAI UE:

sudo RFSIMULATOR=192.168.70.129 ./nr-uesoftmodem -r 106 --numerology 1 --band 78 -C 3619200000 --rfsim --sa --nokrnmod -O ../../../targets/PROJECTS/GENERIC-NR-5GC/CONF/ue.conf --rfsimulator.options chanmod --telnetsrv --channelmod.modellist modellist_rfsimu_1 -d